# Ряды и косинусы

Здесь я напишу функцию, которая умеет вычислять косинус через ряд Тейлора.
Для небольших x работает:

$$\cos x = 1 - \frac{x^2}{2!} + \frac{x^4}{4!} - \frac{x^6}{6!} + \ldots + R(x) = \sum_{n=0}^N \frac{(-1)^n x^{2n}}{(2n)!} + R(x) , x \in \mathbb{C} $$

Это частичная сумма т.н. ряда Тейлора:

$$ f(x) = f(a)+\sum_{k=1}^\infty {f^{(k)} (a) \over k!} (x - a)^k. $$

# Про мой код

В целом, всё закоментированно, но стоит отметить, что вычисление косинуса суммированием ряда Тейлора "в обратном порядке" при очень малых значениях $x$ приводило к непонятным аномалиям, поэтому очень близ нуля будем считать с в "прямом порядке". Можно сказать, что это уже первые костыли :( . Дело в том, что при подсчёте ряда в "прямом порядке" уже при значениях аргумента близ единицы возникает неприятная погрешность, поэтому лучше так, как есть. Тем более, что откидывание всех  $2 \pi k$ превращает график нашего многочлена в приближённый косинус на всей числовой прямой! Позже мы это увидим.

Следующий кусок кода просто создаёт функцию.

In [1]:
#Сразу импортируем модули и библиотеки, которые понадобятся.
import math
import cmath
import numpy
import matplotlib.pyplot as mpp

ITERATIONS = 50

def factorial(a):
    """Лень копаться со встроенными факториалами, напишу свой """
    if a == 0:
        return 1
    return a*factorial(a-1)

def my_cos(x, factorial):
    """
    Вычисление косинуса при помощи частичного суммирования
    ряда Тейлора "в обратном порядке".
    """
    if type(x) != complex and x >= 10**8: #Встроенная самооборона программы
        print('Вы меня убить хотите?')
        return 'Не убивайте :(  введите лучше число поменьше, я всего лишь студенческая программа.'
    
    # Найдём аргумент из полуинтервала [0, 2*pi), эквивалентный введённому
    if type(x) != complex:
        x = abs(x)          # т.к. косинус - чётная функция
        while x > 2*math.pi:
            x -= 2*math.pi
        if x == 0:
            return 1.0
    
    # Найдём последний член, т.е. под номером ITERATIONS+1
    slagaemoe = (((-1)**ITERATIONS)*(x**(2*ITERATIONS))) / (factorial(2*ITERATIONS))
    partial_sum = 0 # Сюда будем запихивать сумму ряда
    if abs(x) >= 0.1: # Иначе аномалии близ нуля, т.к. в цикле я делю на аргумент.
        for n in range(ITERATIONS, 0, -1):
            slagaemoe /= x**2 # Уменьшаем степень
            slagaemoe *= -1 * (2*n-1) * 2*n # Уменьшаем факториал в знаменателе и меняем знак
            partial_sum += slagaemoe # Добавляем слагаемое в сумму
        return partial_sum
    else:             # Близ нуля считаем ряд в "прямом порядке"
        x_pow = 1
        multiplier = 1
        partial_sum = 1
        for n in range(1, ITERATIONS):
            x_pow *= x**2 # Увеличиваем степень
            multiplier *= -1 / (2*n-1) / 2*n # Увеличиваем факториал в знаменателе и меняем знак
            partial_sum += x_pow * multiplier #добавляем слагаемое в сумму
        return partial_sum

help(my_cos)

Help on function my_cos in module __main__:

my_cos(x, factorial)
    Вычисление косинуса при помощи частичного суммирования
    ряда Тейлора "в обратном порядке".



Теперь давайте сравним наш косинус со стандартным библиотечным:

In [ ]:
while True:
    argument=float(input('введите значение аргумента: '))
    print('питоновский косинус:', math.cos(argument))
    print('мой косинус:       ', my_cos(argument, factorial))
    print('')
    if input('Введите bl, если хотите повторить. Иначе press Enter. ') != 'bl':
        break
        

Неплохая точность, хоть и не филигранно. Ну, зато наш косинус умеет и в комплексные числа. Правда здесь уже косинус комплексного аргумента получается точным лишь при малых мнимой и действительной частях (по тестам где-то до 40 работает норм, а дальше уже неточность...).

In [ ]:
while True:
    re_argument = float(input('введите действительную часть аргумента: '))
    im_argument = float(input('введите мнимую часть аргумента: '))
    argument = complex(re_argument, im_argument)
    print('наш аргумент:', argument)
    print('питоновский complex косинус:', cmath.cos(argument))
    print('мой comlex косинус:        ', my_cos(argument, factorial))
    print('')
    if input('Введите bl, если хотите повторить. Иначе press Enter. ') != 'bl':
        break
        

Следующий кусок может удивить искушённого читателя. Оказывается, косинус может достигнуть пяти! Но ведь синус тоже... Ну, мой косинус может достигать даже шести.

In [ ]:
print('')
print('Рассмотрим аргумент:', cmath.acos(6))
print('Косинус из numpy: cos(-2.477888730288475j) =',numpy.cos(-2.477888730288475j))
print('Мой косинус: cos(-2.477888730288475j) =',my_cos(-2.477888730288475j, factorial))
print('Как мы видим, шести косинус достигает.')
print('')

Теперь взглянем на графики.

Сначала нарисуем косинус из библиотеки numpy:

In [ ]:
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'svg')

dlina = float(input('Вы вольны приказать, сколько периодов нарисовать:'))
arguments = numpy.r_[-dlina*math.pi: dlina*math.pi: 0.01]
mpp.title('Косинус из numpy')
mpp.grid()
mpp.xlabel('Аргумент')
mpp.ylabel('Значение')
mpp.plot(arguments, numpy.cos(arguments), color = "blue")
mpp.show()

Теперь нарисуем наш косинус:

In [ ]:
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'svg')

dlina = float(input('Вы вольны приказать, сколько периодов нарисовать:'))
arguments = numpy.r_[-dlina*math.pi: dlina*math.pi: 0.01]
mpp.title('Наш Косинус')
mpp.grid()
mpp.xlabel('Аргумент')
mpp.ylabel('Значение')
mpp.plot(arguments, [my_cos(x, factorial) for x in arguments], color = "red")
mpp.show()

Обратим внимание, что как бы много периодов мы не нарисовали, многочлен нигде не "выстреливает". Это плоды того, что мы откидываем у нашего аргумента все лишние $2 \pi k$

Спасибо за внимание!